In [1]:
import os
import numpy as np
import pandas as pd

os.chdir("../../..")
from xai_green_tech_adoption.utils.utils import *

pd.options.mode.chained_assignment = None

### In this Jupyter notebook we preprocess the owner occupation ratio and variables on the size of households originating from the Zensus 2011.

### 1. Ownership ratio 

In [2]:
# ownership ratio
col_count_owner_occ = "count owner occupied flats"
col_count_all_flats = "total count flats"

In [3]:
df_ownership_raw = pd.read_csv(
    "data/raw_data/descriptive_features/csv_GebaudeWohnungen/Zensus11_Datensatz_Gebaeude.csv",
    sep=";",
    encoding="iso8859_15",
    na_values="-",
    dtype={
        "RS_Land": "str",
        "RS_RB_NUTS2": "str",
        "RS_Kreis": "str",
        "RS_VB": "str",
        "RS_Gem": "str",
    },
    usecols=[
        "RS_Land",
        "RS_RB_NUTS2",
        "RS_Kreis",
        "RS_VB",
        "RS_Gem",
        "Name",
        "WHG_6.1",
        "WHG_6.2",
    ],
)
df_ownership_raw.rename(
    {
        "RS_Land": col_state_code,
        "RS_RB_NUTS2": col_nuts2_code,
        "RS_Kreis": col_county_code,
        "RS_VB": col_ma_code,
        "RS_Gem": col_m_code,
        "Name": col_name_zensus,
        "WHG_6.1": col_count_all_flats,
        "WHG_6.2": col_count_owner_occ,
    },
    axis=1,
    inplace=True,
)
df_ownership_raw.dtypes

state                          object
nuts2 region                   object
county                         object
municipality association       object
municipality                   object
name                           object
total count flats             float64
count owner occupied flats    float64
dtype: object

In [4]:
def preprocess_zensus_feature(
    df_zensus,
    col_m_ags,
    col_state,
    col_nuts2,
    col_county,
    col_m,
    col_feature_denom,
    col_feature_num,
    col_ars,
    name_feature,
):
    """
    Given a raw dataframe (df_zensus) with Zensus data, the function computes the feature 'name_feature' as a fraction of col_feature_num/col_feature_denom (multiplied by 100 as we are interested in the percentage) for all municipality associations. First, I derive the ags for all municipalities by combining the state, nuts2, county and municipality code. Second, the municipalities are mapped to the corresponding municipalitiy associations using 'mapping_municipalities_2011_2019.csv'. Third, I compute the percentage for all municipality associations.
    Assumption: Imprecise values due to privacy issues are indicated as negative.
    @param df_zensus: raw Zensus data
    @param col_m_ags: ags of municipalities
    @param col_state: (string) code of state
    @param col_nuts2: (string) code of (nuts2-) region
    @param col_county: (string) code of county
    @param col_m: (string) code of municipality
    @param col_feature_denom: denominator of feature (count)
    @param col_feature_num: numerator of feature (count)
    @param col_ars: rs of municipality associations
    @param name_feature: name of generated feature
    @return: dataframe encompassing rs of ma's and newly generated feature name_feature
    """

    df_m = df_zensus[df_zensus[col_m].notna()]
    df_m[col_m_ags] = df_m[col_state] + df_m[col_nuts2] + df_m[col_county] + df_m[col_m]
    df_m[col_m_ags] = df_m[col_m_ags].astype(int)

    # Imprecise values are indicated as negative. -> Fix them.
    df_m.loc[df_m[col_feature_num] < 0, col_feature_num] = -1 * df_m[col_feature_num]
    df_m.loc[df_m[col_feature_denom] < 0, col_feature_denom] = (
        -1 * df_m[col_feature_denom]
    )

    # Check for NaNs
    print(
        "There are missing values for the numerator or denominator for the following municipalities. Dropping these municipalities."
    )
    display(df_m[df_m[[col_feature_num, col_feature_denom]].isna().any(axis=1)])
    df_m = df_m[df_m[[col_feature_num, col_feature_denom]].notna().any(axis=1)]

    # map municipalities to corresponding municipality associations
    df_map_m_to_ma = pd.read_csv(
        "data/intermediate_data/mapping_municipalities_2011_2019.csv", sep=";"
    )
    df_m_map = df_m.merge(df_map_m_to_ma, on=col_m_ags, how="left")
    # check if m to ma mapping is available for all m
    if df_m_map[col_ars].isna().sum() == 0:
        print("All municipalities are associated with a municipality association.")
    else:
        print(
            "The m-to-ma-mapping does not provides a municipality association for the following municipalities:"
        )
        display(df_m_map[df_m_map[col_ars].isna()])

    # aggregate data for all municipality associations
    df_agg = pd.DataFrame(
        df_m_map.groupby(by=col_ars, as_index=False).agg(
            {col_feature_num: "sum", col_feature_denom: "sum", col_m_ags: "count"}
        )
    )

    # percentage of flats occupied by owners
    df_agg[name_feature] = 100 * df_agg[col_feature_num] / df_agg[col_feature_denom]
    df_agg[col_ars] = df_agg[col_ars].astype(int)
    return df_agg[[col_ars, name_feature]]

In [5]:
df_owner_agg = preprocess_zensus_feature(
    df_ownership_raw,
    col_m_ags=col_id_m,
    col_state=col_state_code,
    col_nuts2=col_nuts2_code,
    col_county=col_county_code,
    col_m=col_m_code,
    col_feature_denom=col_count_all_flats,
    col_feature_num=col_count_owner_occ,
    col_ars=col_id_ma,
    name_feature=owner_occ_ratio,
)
print('The municipality of Dierfeld belongs to ma Wittlich-Land (jointly with five other municipalities). We drop this municipality.')

There are missing values for the numerator or denominator for the following municipalities. Dropping these municipalities.


,state,nuts2 region,county,municipality association,municipality,name,total count flats,count owner occupied flats,Official municipality code (AGS)
4581,07,2,31,5003,021,Dierfeld,NaN,NaN,7231021


All municipalities are associated with a municipality association.
The municipality of Dierfeld belongs to ma Wittlich-Land (jointly with five other municipalities). We drop this municipality.


In [6]:
def check_completeness_ma(df_to_check, col_id_ma, col_name_ma):
    """
    Check whether df_to_check contains all municipality associations included in the inkar dataset. Function prints result of check.
    @param df_to_check: dataframe to be checked
    @param col_id_ma: rs of municipality associations
    @param col_name_ma: name of municipality associations
    @return: None
    """
    df_inkar_ma_names_ids = pd.read_csv(
        "data/intermediate_data/preprocessed_inkar_data.csv",
        sep=";",
        usecols=[col_id_ma, col_name_ma],
    )
    list_ma_inkar = list(df_inkar_ma_names_ids[col_id_ma])
    list_ma_not_included = [
        ma for ma in list_ma_inkar if ma not in list(df_to_check[col_id_ma])
    ]
    print(
        f"There are {len(list_ma_not_included)} municipality associations that are not included in the ownership occupation data."
    )
    if len(list_ma_not_included) == 0:
        print(
            "All municipality associations of the INKAR dataset are also included in this dataframe."
        )
    else:
        print(
            "Attention: There are municipality associations of the INKAR dataset that are not included in this dataset."
        )
        display(
            df_inkar_ma_names_ids[
                df_inkar_ma_names_ids[col_id_ma].isin(list_ma_not_included)
            ]
        )

In [7]:
check_completeness_ma(df_owner_agg, col_id_ma, col_name_ma)

df_owner_agg.to_csv(
    "data/intermediate_data/preprocessed_ownership_ratio.csv", sep=";", index=False
)

There are 0 municipality associations that are not included in the ownership occupation data.
All municipality associations of the INKAR dataset are also included in this dataframe.


## 2. Household size

In [8]:
col_num_hh_size = [
    col_count_one_p,
    col_count_two_p,
    col_count_three_p,
    col_count_four_p,
    col_count_five_p,
    col_count_six_more_p,
]

In [9]:
df_hh_size_raw = pd.read_csv(
    "data/raw_data/descriptive_features/csv_HaushalteFamilien/Zensus11_Datensatz_Haushalte.csv",
    sep=";",
    encoding="iso8859_15",
    na_values="-",
    dtype={
        "RS_Land": "str",
        "RS_RB_NUTS2": "str",
        "RS_Kreis": "str",
        "RS_Gem": "str",
        "HH_3.1": "str",
        "HH_3.2": "str",
        "HH_3.3": "str",
        "HH_3.4": "str",
        "HH_3.5": "str",
        "HH_3.6": "str",
        "HH_3.7": "str",
    },
    usecols=[
        "RS_Land",
        "RS_RB_NUTS2",
        "RS_Kreis",
        "RS_Gem",
        "Name",
        "HH_3.1",
        "HH_3.2",
        "HH_3.3",
        "HH_3.4",
        "HH_3.5",
        "HH_3.6",
        "HH_3.7",
    ]
)
df_hh_size_raw.rename(
    {
        "RS_Land": col_state_code,
        "RS_RB_NUTS2": col_nuts2_code,
        "RS_Kreis": col_county_code,
        "RS_Gem": col_m_code,
        "Name": col_name_zensus,
        "HH_3.1": col_count_households,
        "HH_3.2": col_count_one_p,
        "HH_3.3": col_count_two_p,
        "HH_3.4": col_count_three_p,
        "HH_3.5": col_count_four_p,
        "HH_3.6": col_count_five_p,
        "HH_3.7": col_count_six_more_p,
    },
    axis=1,
    inplace=True
)

# only consider municipalities
df_hh_m_raw = df_hh_size_raw[df_hh_size_raw[col_m_code].notna()]
df_hh_m_raw[col_id_m] = (
    df_hh_m_raw[col_state_code]
    + df_hh_m_raw[col_nuts2_code]
    + df_hh_m_raw[col_county_code]
    + df_hh_m_raw[col_m_code]
)
df_hh_m_raw[col_id_m] = df_hh_m_raw[col_id_m].astype(int)

# remove parenthesis (incidating imprecise values due to privacy constraints)
df_hh_m_raw[[col_count_households] + col_num_hh_size] = df_hh_m_raw[
    [col_count_households] + col_num_hh_size
].replace({r"\(": "", r"\)": ""}, regex=True)
df_hh_m_raw[[col_count_households] + col_num_hh_size] = df_hh_m_raw[
    [col_count_households] + col_num_hh_size
].astype(float)

# check that dataset does not contain negative values (indicating imprecise values due to privacy constraints)
assert (
    df_hh_m_raw[col_num_hh_size + [col_count_households]] < 0
).sum().sum() == 0, "Attention: Dataframe contains negative values. Check if they indicate imprecise values due to privacy issues."

print('Number of missing values:')
display(df_hh_m_raw[col_num_hh_size + [col_count_households]].isna().sum())
# handle NaNs: check whether the number of households with a specific number of residents sum up to number of all households
# if so, replace NaN by 0
for hh_size in col_num_hh_size:
    df_hh_m_raw.loc[df_hh_m_raw[hh_size].isna(), hh_size] = df_hh_m_raw.apply(
        lambda municipality: 0
        if municipality[col_count_households] == municipality[col_num_hh_size].sum()
        else np.nan,
        axis=1,
    )

print(
    f"There are {df_hh_m_raw[col_num_hh_size].isna().any(axis=1).sum()} remaining municipalities with NaN values."
)
display(df_hh_m_raw[df_hh_m_raw[col_num_hh_size].isna().any(axis=1)])
# drop Dierfeld which is the only m with NaNs (for all household variables)
df_hh_m_raw = df_hh_m_raw[df_hh_m_raw[col_num_hh_size].notna().all(axis=1)]

Number of missing values:


share 1-person households       31
share 2-person households       18
share 3-person households       57
share 4-person households       84
share 5-person households      534
share 6+-person households    1317
count households                 1
dtype: int64

There are 1 remaining municipalities with NaN values.


,state,nuts2 region,county,municipality,name,count households,share 1-person households,share 2-person households,share 3-person households,share 4-person households,share 5-person households,share 6+-person households,Official municipality code (AGS)
4581,07,2,31,021,Dierfeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7231021


In [10]:
df_hh_m = df_hh_m_raw[[col_id_m, col_count_households] + col_num_hh_size]

df_map_m_to_ma = pd.read_csv(
    "data/intermediate_data/mapping_municipalities_2011_2019.csv", sep=";"
)
df_hh_m = df_hh_m.merge(df_map_m_to_ma, on=col_id_m, how="left")

# check if there is an ma assignes to each m
assert (
    df_hh_m[col_id_ma].isna().sum() == 0
), "Attention: There is no assignes ma for some m."

df_hh_ma = pd.DataFrame(df_hh_m.groupby(by=col_id_ma, as_index=False).sum())

for hh_size in col_num_hh_size:
    df_hh_ma[hh_size] = df_hh_ma[hh_size] / df_hh_ma[col_count_households] * 100
df_hh_ma.drop([col_id_m], inplace=True, axis=1)

# check that dataframe covers all ma's given by inkar
check_completeness_ma(df_hh_ma, col_id_ma, col_name_ma)

df_hh_ma.to_csv(
    "data/intermediate_data/preprocessed_household_sizes_with_hh_counts.csv",
    sep=";",
    index=False,
)

There are 0 municipality associations that are not included in the ownership occupation data.
All municipality associations of the INKAR dataset are also included in this dataframe.
